In [ ]:
# DataSpark : Illuminating Insights for Global Electronics

In [1]:
pip install pandas

  Using cached pandas-2.2.2-cp312-cp312-win_amd64.whl.metadata (19 kB)
  Using cached pytz-2024.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.2-cp312-cp312-win_amd64.whl (11.5 MB)
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   -------------------- ------------------- 6.3/12.6 MB 32.2 MB/s eta 0:00:01
   ---------------------------------------  12.3/12.6 MB 30.8 MB/s eta 0:00:01
   ---------------------------------------- 12.6/12.6 MB 29.2 MB/s eta 0:00:00
Using cached pytz-2024.1-py2.py3-none-any.whl (505 kB)
Using cached tzdata-2024.1-py2.py3-none-any.whl (345 kB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd

# Load dataset
customers = pd.read_csv('CSV files\Customers.csv', encoding='ISO-8859-1')

<>:4: SyntaxWarning: invalid escape sequence '\C'
<>:4: SyntaxWarning: invalid escape sequence '\C'
C:\Users\WELCOME\AppData\Local\Temp\ipykernel_3244\3013831896.py:4: SyntaxWarning: invalid escape sequence '\C'
  customers = pd.read_csv('CSV files\Customers.csv', encoding='ISO-8859-1')


In [3]:
customers

,CustomerKey,Gender,Name,City,State Code,State,Zip Code,Country,Continent,Birthday
0,301,Female,Lilly Harding,WANDEARAH EAST,SA,South Australia,5523,Australia,Australia,7/3/1939
1,325,Female,Madison Hull,MOUNT BUDD,WA,Western Australia,6522,Australia,Australia,9/27/1979
2,554,Female,Claire Ferres,WINJALLOK,VIC,Victoria,3380,Australia,Australia,5/26/1947
3,786,Male,Jai Poltpalingada,MIDDLE RIVER,SA,South Australia,5223,Australia,Australia,9/17/1957
4,1042,Male,Aidan Pankhurst,TAWONGA SOUTH,VIC,Victoria,3698,Australia,Australia,11/19/1965
...,...,...,...,...,...,...,...,...,...,...
15261,2099600,Female,Denisa Duková,Houston,TX,Texas,77017,United States,North America,3/25/1936
15262,2099618,Male,Justin Solórzano,Mclean,VA,Virginia,22101,United States,North America,2/16/1992
15263,2099758,Male,Svend Petrussen,Wilmington,NC,North Carolina,28405,United States,North America,11/9/1937
15264,2099862,Female,Lorenza Rush,Riverside,CA,California,92501,United States,North America,10/12/1937


In [4]:
# Handling Missing Values
customers['City'].fillna('Unknown', inplace=True)
customers['State'].fillna('Unknown', inplace=True)
customers['State Code'].fillna('Unknown', inplace=True)  
customers['Zip Code'].fillna(0, inplace=True)
customers['Country'].fillna('Unknown', inplace=True)
customers['Continent'].fillna('Unknown', inplace=True)

# Correcting Data Types
customers['Birthday'] = pd.to_datetime(customers['Birthday'], errors='coerce')

# Removing Duplicates (excluding primary key and unique key)
customers.drop_duplicates(subset=[col for col in customers.columns if col not in ['CustomerKey']], inplace=True)

# Renaming Columns for Consistency
customers.rename(columns={'CustomerKey': 'Customer_ID', 'Zip Code': 'Zip_Code', 'State Code': 'State_Code'}, inplace=True)

# Handling Categorical Data
customers['Gender'] = customers['Gender'].astype('category')

# Calculate Age
customers['Age'] = (pd.Timestamp.now() - customers['Birthday']).dt.total_seconds() / (60*60*24*365.25)
customers['Age'] = customers['Age'].astype(int)

customers

C:\Users\WELCOME\AppData\Local\Temp\ipykernel_3244\3606214188.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  customers['City'].fillna('Unknown', inplace=True)
C:\Users\WELCOME\AppData\Local\Temp\ipykernel_3244\3606214188.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For 

,Customer_ID,Gender,Name,City,State_Code,State,Zip_Code,Country,Continent,Birthday,Age
0,301,Female,Lilly Harding,WANDEARAH EAST,SA,South Australia,5523,Australia,Australia,1939-07-03,85
1,325,Female,Madison Hull,MOUNT BUDD,WA,Western Australia,6522,Australia,Australia,1979-09-27,44
2,554,Female,Claire Ferres,WINJALLOK,VIC,Victoria,3380,Australia,Australia,1947-05-26,77
3,786,Male,Jai Poltpalingada,MIDDLE RIVER,SA,South Australia,5223,Australia,Australia,1957-09-17,66
4,1042,Male,Aidan Pankhurst,TAWONGA SOUTH,VIC,Victoria,3698,Australia,Australia,1965-11-19,58
...,...,...,...,...,...,...,...,...,...,...,...
15261,2099600,Female,Denisa Duková,Houston,TX,Texas,77017,United States,North America,1936-03-25,88
15262,2099618,Male,Justin Solórzano,Mclean,VA,Virginia,22101,United States,North America,1992-02-16,32
15263,2099758,Male,Svend Petrussen,Wilmington,NC,North Carolina,28405,United States,North America,1937-11-09,86
15264,2099862,Female,Lorenza Rush,Riverside,CA,California,92501,United States,North America,1937-10-12,86


In [6]:

# Save cleaned dataset
customers.to_csv('idealprocessed/customers.csv', index=False)

print("Customers data cleaning completed.")



Customers data cleaning completed.


In [10]:
import pandas as pd
import numpy as np

# Load dataset
sales = pd.read_csv('CSV files\Sales.csv', encoding='ISO-8859-1')

<>:5: SyntaxWarning: invalid escape sequence '\S'
<>:5: SyntaxWarning: invalid escape sequence '\S'
C:\Users\WELCOME\AppData\Local\Temp\ipykernel_3244\1967192741.py:5: SyntaxWarning: invalid escape sequence '\S'
  sales = pd.read_csv('CSV files\Sales.csv', encoding='ISO-8859-1')


In [11]:
sales

,Order Number,Line Item,Order Date,Delivery Date,CustomerKey,StoreKey,ProductKey,Quantity,Currency Code
0,366000,1,1/1/2016,NaN,265598,10,1304,1,CAD
1,366001,1,1/1/2016,1/13/2016,1269051,0,1048,2,USD
2,366001,2,1/1/2016,1/13/2016,1269051,0,2007,1,USD
3,366002,1,1/1/2016,1/12/2016,266019,0,1106,7,CAD
4,366002,2,1/1/2016,1/12/2016,266019,0,373,1,CAD
...,...,...,...,...,...,...,...,...,...
62879,2243030,1,2/20/2021,NaN,1216913,43,632,3,USD
62880,2243031,1,2/20/2021,2/24/2021,511229,0,98,4,EUR
62881,2243032,1,2/20/2021,2/23/2021,331277,0,1613,2,CAD
62882,2243032,2,2/20/2021,2/23/2021,331277,0,1717,2,CAD


In [12]:
# Define primary and unique key columns
primary_keys_sales = ['Order Number', 'Line Item', 'CustomerKey', 'StoreKey', 'ProductKey']

# Handling Missing Values
sales.dropna(subset=['Order Date'], inplace=True)

# Correcting Data Types
sales['Order Date'] = pd.to_datetime(sales['Order Date'], errors='coerce')

# Handling Outliers (e.g., capping extreme values in Quantity)
sales['Quantity'] = np.where(sales['Quantity'] > sales['Quantity'].quantile(0.99), sales['Quantity'].quantile(0.99), sales['Quantity'])

# Removing Duplicates
sales.drop_duplicates(inplace=True)

# High Percentage of Missing Values
threshold = 0.5
columns_to_drop = [col for col in sales.columns if sales[col].isnull().sum() / len(sales) > threshold and col not in primary_keys_sales]

# Drop irrelevant, low variance, redundant, high correlation, and privacy-sensitive columns as per conditions
sales.drop(columns=set(columns_to_drop), inplace=True)

# Save cleaned dataset
sales.to_csv('idealprocessed/sales.csv', index=False)

print("Sales data cleaning completed.")


Sales data cleaning completed.


In [13]:
import pandas as pd

# Load dataset
stores = pd.read_csv('CSV files\Stores.csv', encoding='ISO-8859-1')

# Display the first few rows to understand the data
print(stores.head())

# Handling Missing Values

stores['Country'].fillna('Unknown', inplace=True)
stores['State'].fillna('Unknown', inplace=True)
stores['Square Meters'].fillna(0, inplace=True)

# Correcting Data Types
stores['Open Date'] = pd.to_datetime(stores['Open Date'], errors='coerce')

# Removing Duplicates (excluding primary key)
stores.drop_duplicates(subset=[col for col in stores.columns if col not in ['StoreKey']], inplace=True)

# Handle Inconsistent Data (if any)

# Save cleaned dataset
stores.to_csv('idealprocessed/stores.csv', index=False)

print("Stores data cleaning completed.")
stores


   StoreKey    Country                         State  Square Meters  Open Date
0         1  Australia  Australian Capital Territory          595.0   1/1/2008
1         2  Australia            Northern Territory          665.0  1/12/2008
2         3  Australia               South Australia         2000.0   1/7/2012
3         4  Australia                      Tasmania         2000.0   1/1/2010
4         5  Australia                      Victoria         2000.0  12/9/2015
Stores data cleaning completed.


<>:4: SyntaxWarning: invalid escape sequence '\S'
<>:4: SyntaxWarning: invalid escape sequence '\S'
C:\Users\WELCOME\AppData\Local\Temp\ipykernel_3244\1569395229.py:4: SyntaxWarning: invalid escape sequence '\S'
  stores = pd.read_csv('CSV files\Stores.csv', encoding='ISO-8859-1')
C:\Users\WELCOME\AppData\Local\Temp\ipykernel_3244\1569395229.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  stores['Country'].fillna('Unknown', inplace=True)
C:\Users\WELCOME\AppData\Local\Temp\ipykernel_3244\1569395229.py:12: FutureWarning: A

,StoreKey,Country,State,Square Meters,Open Date
0,1,Australia,Australian Capital Territory,595.0,2008-01-01
1,2,Australia,Northern Territory,665.0,2008-01-12
2,3,Australia,South Australia,2000.0,2012-01-07
3,4,Australia,Tasmania,2000.0,2010-01-01
4,5,Australia,Victoria,2000.0,2015-12-09
...,...,...,...,...,...
62,63,United States,Utah,2000.0,2008-03-06
63,64,United States,Washington DC,1330.0,2010-01-01
64,65,United States,West Virginia,1785.0,2012-01-01
65,66,United States,Wyoming,840.0,2014-01-01


In [14]:
import pandas as pd

# Load dataset
products = pd.read_csv('CSV files\Products.csv', encoding='ISO-8859-1')

# Handling Missing Values
products['Color'].fillna('Unknown', inplace=True)

# Removing Duplicates
products.drop_duplicates(inplace=True)

# Remove currency symbols before converting to numeric
for column in ['Unit Cost USD', 'Unit Price USD']:
    products[column] = products[column].str.replace('$', '').str.replace(',', '')

# Correcting Data Types (e.g., converting cost and price to numeric)
for column in ['Unit Cost USD', 'Unit Price USD']:
    try:
        products[column] = pd.to_numeric(products[column], errors='coerce')
        print(f"Converted {column} to numeric successfully.")
    except Exception as e:
        print(f"Error converting {column} to numeric: {e}")

# Additional step to handle missing values in cost and price columns
for column in ['Unit Cost USD', 'Unit Price USD']:
    if products[column].isnull().any():
        products[column].fillna(products[column].median(), inplace=True)
        print(f"Filled missing values in {column} with the median.")

# Save cleaned dataset
products.to_csv('idealprocessed/products.csv', index=False)

print("Products data cleaning completed.")


Converted Unit Cost USD to numeric successfully.
Converted Unit Price USD to numeric successfully.
Products data cleaning completed.


<>:4: SyntaxWarning: invalid escape sequence '\P'
<>:4: SyntaxWarning: invalid escape sequence '\P'
C:\Users\WELCOME\AppData\Local\Temp\ipykernel_3244\2700221647.py:4: SyntaxWarning: invalid escape sequence '\P'
  products = pd.read_csv('CSV files\Products.csv', encoding='ISO-8859-1')
C:\Users\WELCOME\AppData\Local\Temp\ipykernel_3244\2700221647.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  products['Color'].fillna('Unknown', inplace=True)


In [15]:
import pandas as pd

# Load dataset
exchange_rates = pd.read_csv('CSV files\Exchange_Rates.csv', encoding='ISO-8859-1')

# Correcting Data Types
exchange_rates['Date'] = pd.to_datetime(exchange_rates['Date'], errors='coerce')

# Removing Duplicates
exchange_rates.drop_duplicates(inplace=True)

# Save cleaned dataset
exchange_rates.to_csv('idealprocessed/exchangerates.csv', index=False)

print("Exchange rates data cleaning completed.")


Exchange rates data cleaning completed.


<>:4: SyntaxWarning: invalid escape sequence '\E'
<>:4: SyntaxWarning: invalid escape sequence '\E'
C:\Users\WELCOME\AppData\Local\Temp\ipykernel_3244\2863279714.py:4: SyntaxWarning: invalid escape sequence '\E'
  exchange_rates = pd.read_csv('CSV files\Exchange_Rates.csv', encoding='ISO-8859-1')


In [16]:
import pandas as pd

# Load cleaned datasets
customers = pd.read_csv('idealprocessed\customers.csv', encoding='ISO-8859-1')
sales = pd.read_csv('idealprocessed\sales.csv', encoding='ISO-8859-1')
stores = pd.read_csv('idealprocessed\stores.csv', encoding='ISO-8859-1')
products = pd.read_csv('idealprocessed\products.csv', encoding='ISO-8859-1')
exchange_rates = pd.read_csv('idealprocessed\exchangerates.csv', encoding='ISO-8859-1')

# Merge sales with customers
sales_customers = pd.merge(sales, customers, how='left', left_on='CustomerKey', right_on='Customer_ID')

# Merge sales_customers with stores
sales_customers_stores = pd.merge(sales_customers, stores, how='left', left_on='StoreKey', right_on='StoreKey')

# Merge sales_customers_stores with products
sales_customers_stores_products = pd.merge(sales_customers_stores, products, how='left', left_on='ProductKey', right_on='ProductKey')

# Optionally, merge with exchange rates if needed (e.g., if there are columns that need exchange rates)
# Assuming you might want to join on a common date column, which isn't directly mentioned in the given datasets
# sales_customers_stores_products = pd.merge(sales_customers_stores_products, exchange_rates, how='left', left_on='Date', right_on='Date')

# Save the merged dataset
sales_customers_stores_products.to_csv('idealprocessed/merged.csv', index=False)

print("Data merging completed.")


<>:4: SyntaxWarning: invalid escape sequence '\c'
<>:5: SyntaxWarning: invalid escape sequence '\s'
<>:6: SyntaxWarning: invalid escape sequence '\s'
<>:7: SyntaxWarning: invalid escape sequence '\p'
<>:8: SyntaxWarning: invalid escape sequence '\e'
<>:4: SyntaxWarning: invalid escape sequence '\c'
<>:5: SyntaxWarning: invalid escape sequence '\s'
<>:6: SyntaxWarning: invalid escape sequence '\s'
<>:7: SyntaxWarning: invalid escape sequence '\p'
<>:8: SyntaxWarning: invalid escape sequence '\e'
C:\Users\WELCOME\AppData\Local\Temp\ipykernel_3244\3266425439.py:4: SyntaxWarning: invalid escape sequence '\c'
  customers = pd.read_csv('idealprocessed\customers.csv', encoding='ISO-8859-1')
C:\Users\WELCOME\AppData\Local\Temp\ipykernel_3244\3266425439.py:5: SyntaxWarning: invalid escape sequence '\s'
  sales = pd.read_csv('idealprocessed\sales.csv', encoding='ISO-8859-1')
C:\Users\WELCOME\AppData\Local\Temp\ipykernel_3244\3266425439.py:6: SyntaxWarning: invalid escape sequence '\s'
  stores =

Data merging completed.


In [17]:
pip install pandas sqlalchemy mysql-connector-python


  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 2.1/2.1 MB 28.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/14.3 MB ? eta -:--:--
   --------------- ------------------------ 5.5/14.3 MB 30.5 MB/s eta 0:00:01
   ----------------------------- ---------- 10.7/14.3 MB 24.8 MB/s eta 0:00:01
   ---------------------------------------- 14.3/14.3 MB 24.4 MB/s eta 0:00:00
Using cached typing_extensions-4.12.2-py3-none-any.whl (37 kB)
Note: you may need to restart the kernel to use updated packages.


In [18]:
import pandas as pd
from sqlalchemy import create_engine


file_path = 'idealprocessed\merged.csv'  # Path to your uploaded file
df = pd.read_csv(file_path)


user = 'root'
password = 'root'
host = 'localhost'  # Replace with your MySQL host if not localhost
database = 'dataspark'

# Create connection string
connection_string = f'mysql+mysqlconnector://{user}:{password}@{host}/{database}'
engine = create_engine(connection_string)

table_name = 'mergeddata'
df.to_sql(table_name, con=engine, if_exists='replace', index=False)

<>:5: SyntaxWarning: invalid escape sequence '\m'
<>:5: SyntaxWarning: invalid escape sequence '\m'
C:\Users\WELCOME\AppData\Local\Temp\ipykernel_3244\2444957660.py:5: SyntaxWarning: invalid escape sequence '\m'
  file_path = 'idealprocessed\merged.csv'  # Path to your uploaded file


62884

In [63]:
import sqlite3
import pandas as pd

# Paths to the cleaned datasets
customers_path = 'D:\\Guvi\\projects\\Data Spark\\cleaned_customers.csv'
sales_path = 'D:\\Guvi\\projects\\Data Spark\\cleaned_sales.csv'
stores_path = 'D:\\Guvi\\projects\\Data Spark\\cleaned_stores.csv'
products_path = 'D:\\Guvi\\projects\\Data Spark\\cleaned_products.csv'
exchange_rates_path = 'D:\\Guvi\\projects\\Data Spark\\cleaned_exchange_rates.csv'
merged_data_path = 'D:\\Guvi\\projects\\Data Spark\\merged_data.csv'

# Path to the SQLite database
db_path = 'D:\\Guvi\\projects\\Data Spark\\Data_Spark.db'

# Connect to the SQLite database
conn = sqlite3.connect(db_path)

# Load each dataset
customers = pd.read_csv(customers_path)
sales = pd.read_csv(sales_path)
stores = pd.read_csv(stores_path)
products = pd.read_csv(products_path)
exchange_rates = pd.read_csv(exchange_rates_path)
merged_data = pd.read_csv(merged_data_path)

# Insert each DataFrame into the SQLite database
customers.to_sql('Customers', conn, if_exists='replace', index=False)
sales.to_sql('Sales', conn, if_exists='replace', index=False)
stores.to_sql('Stores', conn, if_exists='replace', index=False)
products.to_sql('Products', conn, if_exists='replace', index=False)
exchange_rates.to_sql('ExchangeRates', conn, if_exists='replace', index=False)
merged_data.to_sql('MergedData', conn, if_exists='replace', index=False)

# Close the connection
conn.close()

print("All datasets successfully stored in the SQLite database.")


All datasets successfully stored in the SQLite database.
